# Structs Introduction

A struct, or structure, is a custom data type that <span style="color:lightgreen">*lets you package together and name multiple related values*</span> that make up a meaningful group.  
We will also learn <span style="color:lightgreen">*struct associated functions, especially the kind of associated functions called methods*</span>, to specify behavior associated with a struct type

# Defining and Instantiating Structs

- **Structs vs Tuples**: Like tuples, the pieces of a struct can be different types. Unlike with tuples, in a struct you’ll name each piece of data so it’s clear what the values mean. We define a struct and its fields like below

In [2]:
struct User {
    active: bool,  // each piece of data is called 'field'
    username: String,
    email: String,
    sign_in_count: u64,
}


Struct Instance: To use a struct after we’ve defined it, we *create an instance of that struct* by specifying concrete values for each of the fields. 

In [5]:
let user1: User = User {
        active: true,
        username: String::from("someusername123"),
        email: String::from("someone@example.com"),
        sign_in_count: 1,
};

Field Accessing: To get a specific value from a struct, we use dot notation. For example, to access this user’s email address, we use `user1.email`. If the instance is mutable, we can change a value by using the dot notation and assigning into a particular field.

In [10]:
fn main() {
    let mut user1 = User {
        active: true,
        username: String::from("someusername123"),
        email: String::from("someone@example.com"),
        sign_in_count: 1,
    };

    user1.email = String::from("anotheremail@example.com");
    println!("{}", user1.email);
}

main()

anotheremail@example.com


()

Note that the entire instance must be mutable; Rust doesn’t allow us to mark only certain fields as mutable.

## Using the Field Init Shorthand

In [14]:
fn build_user(email: String, username: String) -> User {
    User {
        active: true,
        username,  // same with username: username
        email,  // same with email: email
        sign_in_count: 1,
    }
}

let a = build_user(String::from("anotheremail@example.com"), String::from("user1"));

## Creating Instances from Other Instances with Struct Update Syntax

Look at how we create the `user2` below, where the syntax `..` specifies that the remaining fields not explicitly set should have the same value as the fields in the given instance.

In [16]:
struct User {
    active: bool,
    username: String,
    email: String,
    sign_in_count: u64,
}

fn main() {
    // --snip--

    let user1 = User {
        email: String::from("user1@example.com"),
        username: String::from("someusername123"),
        active: true,
        sign_in_count: 1,
    };

    let user2 = User {
        email: String::from("another@example.com"),
        ..user1  // this must come last
    };
}

> 📙 **Note**  
> <span style="color:orange">*Note that **the struct update syntax uses `=` like an assignment and it moves the data, just as we saw in the “Variables and Data Interacting with Move” section**. In this example, we can no longer use `user1` as a whole after creating `user2` because the `String` in the username field of `user1` was moved into `user2`. If we had given `user2` new `String` values for both `email` and `username`, and thus only used the `active` and `sign_in_count` values from `user1`, then `user1` would still be valid after creating `user2` since `active` and `sign_in_count` are types that implement the `Copy` trait, so they are copied into `user2`. However, `String` does not implement `Copy` and was moved into `user2`* </span>

## Using Tuple Structs Without Named Fields to Create Different Types

<span style="color:lightgreen">*Rust also supports structs that look similar to tuples, called tuple structs. **Tuple structs have the added meaning the struct name provides but don’t have names associated with their fields**; rather, they just have the types of the fields.*</span>

In [18]:
struct Color(i32, i32, i32);
struct Point(i32, i32, i32);

let black = Color(0, 0, 0);
let origin = Point(0, 0, 0);

Note that `black` and `origin` are instances of different types, hence, a function that takes a parameter of type `Color` cannot take a `Point` as an argument, even though both types are made up of three `i32` values.

## Unit-Like Structs Without Any Fields

<span style="color:lightgreen">***Unit-like structs** are structs that don’t have any fields and can useful when you need to implement a trait on some type but don’t have any data that you want to store in the type itself.*</span>

In [ ]:
struct AlwaysEqual;

fn main() {
    // Imagine that later we’ll implement behavior for this type such that every instance 
    // of AlwaysEqual is always equal to every instance of any other type, perhaps to 
    // have a known result for testing purposes
    let subject = AlwaysEqual;
}

## Ownership of Struct Data
Why in 
```rust
struct User {
    active: bool,  // each piece of data is called 'field'
    username: String,
    email: String,
    sign_in_count: u64,
}
```
we define `username` and `email` as `String`, not `&str`?  
<span style="color:skyblue">*Because we want each instance of this struct to own all of its data and for that data to be valid for as long as the entire struct is valid.*</span>

<span style="color:orange">*It’s also possible for structs to store references to data owned by something else, but to do so requires the use of **lifetimes**. **Lifetimes ensure that the data referenced by a struct is valid for as long as the struct is**. Let’s say you try to store a reference in a struct without specifying lifetimes, like the following; this won’t work and produce the error `missing lifetime specifier`:*</span>

In [19]:
struct User {
    active: bool,
    username: &str,
    email: &str,
    sign_in_count: u64,
}

fn main() {
    let user1 = User {
        active: true,
        username: "someusername123",
        email: "someone@example.com",
        sign_in_count: 1,
    };
}

Error: missing lifetime specifier

Error: missing lifetime specifier

Error: mismatched types

Error: mismatched types

# An Example Program Using Struct

In [2]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

fn main() {
    let scale = 2;
    let rect1 = Rectangle {
        height: 50,
        width: dbg!(30 * scale), // Note that dbg! macro takes ownership of an expression
                                 // Also, dbg! returns ownership of the expression’s value
                                 // so width will have the value of 60
    };

    dbg!(&rect1);

    println!("The area is {}", area(&rect1))
}

fn area(rec: &Rectangle) -> u32 {
    rec.width * rec.height
}

main()

The area is 3000


[src/lib.rs:6] 30 * scale = 60
[src/lib.rs:11] &rect1 = Rectangle {
    width: 60,
    height: 50,
}


()

Our `area` function is very specific: it only computes the `area` of rectangles and won’t work with any other type.  
Let’s look at how we tight the `area` function into an `area` method defined on only our `Rectangle` type.

# Method Syntax

<span style="color:lightgreen">**Methods are similar to functions, but they are defined within the context of a struct (or an enum or a trait object), and their first parameter is always `self`, which represents the instance of the struct the method is being called on.**</span>

## Defining Methods

Define methods of a struct using `impl` like below

In [15]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

impl Rectangle {
    fn area(&self) -> u32 {
        self.width * self.height
    }

    fn print(&self) {
        println!("The rectangle has width = {}, height = {}.", self.width, self.height);
    }

    fn change_width(&mut self, new_width: u32) {
        self.width = new_width;
    }

    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width > other.width && self.height > other.height
    }
}

fn main() {
    let mut rect1 = Rectangle {
        width: 30,
        height: 50,
    };

    let rect2 = Rectangle {
        width: 20,
        height: 30,
    };
    
    rect1.print();
    println!(
        "The area of the rectangle is {} square pixels.",
        rect1.area()
    );
    println!("Can rect1 hold rect2? {}", rect1.can_hold(&rect2));

    println!("Changing the width of rect1");
    rect1.change_width(15);
    rect1.print();
    println!("Can rect1 hold rect2? {}", rect1.can_hold(&rect2));
    println!(
        "The area of the rectangle is {} square pixels.",
        rect1.area()
    );


}

main()

The rectangle has width = 30, height = 50.
The area of the rectangle is 1500 square pixels.
Can rect1 hold rect2? true
Changing the width of rect1
The rectangle has width = 15, height = 50.
Can rect1 hold rect2? false
The area of the rectangle is 750 square pixels.


()

> **Notes on `&self`**
>
> In the signature for area, we use `&self` instead of `rectangle: &Rectangle`. <span style="color:lightgreen">**The `&self` is actually short for `self: &Self`**</span>. Within an `impl` block, the type `Self` is an alias for the type that the `impl` block is for. Methods must have a parameter named `self` of type `Self` for their first parameter, so Rust lets you abbreviate this with only the name `self` in the first parameter spot. Note that we still need to use the `&` in front of the self shorthand to indicate that this method borrows the `Self` instance, just as we did in `rectangle: &Rectangle`.   
> <span style="color:lightgreen">***Methods can take ownership of `self` (by using just `self` as the first parameter, but this is very rare), borrow `self` immutably, as we’ve done here, or borrow `self` mutably, just as they can any other parameter.***</span>

## Where’s the `->` Operator?

> <span style="color:skyblue">*In C and C++, you use `.` if you’re calling a method on the object directly and `->` if you’re calling the method on a pointer to the object and need to dereference the pointer first*</span>. In other words, if object is a pointer, `object->something()` is similar to `(*object).something()`.
>
> <span style="color:skyblue">***Rust doesn’t have an equivalent to the `->` operator; instead, Rust has a feature called automatic referencing and dereferencing***</span>. Calling methods is one of the few places in Rust that has this behavior. <span style="color:lightgreen">***When you call a method with `object.something()`, Rust automatically adds in `&`, `&mut`, or `*` so object matches the signature of the method***</span>. In other words, the following are the same:
> ```rust
> p1.distance(&p2);
> (&p1).distance(&p2);
> ```
> The first one looks much cleaner. This automatic referencing behavior works because methods have a clear receiver—the type of `self`. Given the receiver and name of a method, Rust can figure out definitively whether the method is reading (`&self`), mutating (`&mut self`), or consuming (`self`). The fact that Rust makes borrowing implicit for method receivers is a big part of making ownership ergonomic in practice.

## Associated Functions

All functions defined within an `impl` block are called *associated functions* because they’re associated with the type named after the `impl`. <span style="color:lightgreen">***We can define associated functions that don’t have `self` as their first parameter (and thus are not methods) because they don’t need an instance of the type to work with. This is similar to Python's `@classmethod`.***</span>. We’ve already used one function like this: the `String::from` function that’s defined on the `String` type.

In [17]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

impl Rectangle {
    fn square(size: u32) -> Self {
        Self {
            width: size,
            height: size,
        }
    }
}

fn main() {
    let sq = Rectangle::square(3);
}

<span style="color:yellow">Note that the `::` syntax is used for both associated functions and namespaces created by modules</span>

## Multiple `impl` Blocks

Each struct is allowed to have multiple `impl` blocks.

In [20]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

impl Rectangle {
    fn area(&self) -> u32 {
        self.width * self.height
    }
}

impl Rectangle {
    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width > other.width && self.height > other.height
    }
}

fn main() {
    let rect1 = Rectangle {
        width: 30,
        height: 50,
    };
    let rect2 = Rectangle {
        width: 10,
        height: 40,
    };
    let rect3 = Rectangle {
        width: 60,
        height: 45,
    };

    println!("Can rect1 hold rect2? {}", rect1.can_hold(&rect2));
    println!("Can rect1 hold rect3? {}", rect1.can_hold(&rect3));
}

main()

Can rect1 hold rect2? true
Can rect1 hold rect3? false


()